## ESMA 4016
## Edgar Acuna
## Imputation II
### Imputacion por la media media y moda y usando knn en los datasests: Titanic y segment 

In [5]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as sm
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import preprocessing 

In [6]:
#Ejemplo1. Leyendo los datos de Titanic
titanic=pd.read_csv('http://academic.uprm.edu/eacuna/titanic.csv',header=0,sep=',',na_values='')
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 11 columns):
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Name           1309 non-null object
Sex            1309 non-null object
Age            1046 non-null float64
SibSp          1309 non-null int64
Parch          1309 non-null int64
Ticket         1309 non-null object
Fare           1308 non-null float64
Cabin          295 non-null object
Embarked       1307 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 112.6+ KB


In [7]:
#Eiminando las columnas PassengerId, Name y cabin(la cual tiene demasiados
#missing values)
titanic1=titanic.iloc[:,[1,3,4,5,6,7,8,10]]
titanic1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 8 columns):
Pclass      1309 non-null int64
Sex         1309 non-null object
Age         1046 non-null float64
SibSp       1309 non-null int64
Parch       1309 non-null int64
Ticket      1309 non-null object
Fare        1308 non-null float64
Embarked    1307 non-null object
dtypes: float64(2), int64(3), object(3)
memory usage: 81.9+ KB


In [8]:
#Imputando los missing values por la media si el atributo es continuo
#y por la moda si el atributo es nominal
from sklearn.preprocessing import Imputer
from sklearn.base import TransformerMixin

class DataFrameImputer(TransformerMixin):

    def fit(self, X, y=None):

        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].mean() for c in X],
            index=X.columns)

        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)
X=titanic1
xt = DataFrameImputer().fit_transform(X)

In [9]:
xt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 8 columns):
Pclass      1309 non-null int64
Sex         1309 non-null object
Age         1309 non-null float64
SibSp       1309 non-null int64
Parch       1309 non-null int64
Ticket      1309 non-null object
Fare        1309 non-null float64
Embarked    1309 non-null object
dtypes: float64(2), int64(3), object(3)
memory usage: 81.9+ KB


In [10]:
#Imputando los missing values por la mediana si el atributo es continuo
#y por la moda si el atributo es nominal

class DataFrameImputer(TransformerMixin):

    def fit(self, X, y=None):

        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].median() for c in X],
            index=X.columns)

        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)
X=titanic1
xt = DataFrameImputer().fit_transform(X)

In [11]:
#imputando por knn
from sklearn import datasets
from sklearn import neighbors

class ImputeLearn(object):
    def __init__(self, mat):
        self.mat = np.array(mat)
        self.mat_isnan = np.isnan(self.mat)
        self.w = np.where(np.isnan(self.mat))

    def impute(self, learner):
        ximp = self.mat.copy()
        for i in np.arange(0, len(self.w[0])):
            n = self.w[0][i] # row where the nan is
            p = self.w[1][i] # column where the nan is
            col_isnan = self.mat_isnan[n, :] # empty columns in row n
            train = np.delete(self.mat, n, axis = 0) # remove row n to obtain a training set
            train_nonan = train[~np.apply_along_axis(np.any, 1, np.isnan(train)), :] # remove rows where there is a nan in the training set
            target = train_nonan[:, p] # vector to be predicted
            feature = train_nonan[:, ~col_isnan] # matrix of predictors
            learn = learner.fit(feature, target) # learner
            ximp[n, p] = learn.predict(self.mat[n, ~col_isnan].reshape(1, -1)) # predict and replace
        
        return(ximp)

In [12]:
#Convirtiendo las variables nominales en numericas, puesto que 
#la imputacion knn de python no trabaja con variables mezcladas
titanic2=titanic1.apply(preprocessing.LabelEncoder().fit_transform)

In [13]:
titanic2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 8 columns):
Pclass      1309 non-null int64
Sex         1309 non-null int64
Age         1309 non-null int64
SibSp       1309 non-null int64
Parch       1309 non-null int64
Ticket      1309 non-null int64
Fare        1309 non-null int64
Embarked    1309 non-null int64
dtypes: int64(8)
memory usage: 81.9 KB


In [14]:
titanic2.head()

,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,2,1,32,1,0,720,20,4
1,0,0,57,1,0,816,237,2
2,2,0,39,0,0,914,50,4
3,0,0,53,1,0,65,217,4
4,2,1,53,0,0,649,52,4


In [15]:
b1=np.array(titanic2)

In [16]:
titanic_knnimp=ImputeLearn(b1).impute(learner = neighbors.KNeighborsRegressor(n_neighbors = 5))

In [17]:
completo=pd.DataFrame(titanic_knnimp)
completo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 8 columns):
0    1309 non-null int64
1    1309 non-null int64
2    1309 non-null int64
3    1309 non-null int64
4    1309 non-null int64
5    1309 non-null int64
6    1309 non-null int64
7    1309 non-null int64
dtypes: int64(8)
memory usage: 81.9 KB


In [18]:
#Ejemplo 2. Simulando misising values e imputandolos wn wl dataset segment de la UCI
seg=pd.read_table('http://archive.ics.uci.edu/ml/machine-learning-databases/statlog/segment/segment.dat',header=None, delim_whitespace=True)
seg.columns=['v1','v2','v3','v4','v5','v6','v7','v8','v9','v10','v11','v12','v13','v14','v15','v16','v17','v18','v19','class']

In [19]:
seg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2310 entries, 0 to 2309
Data columns (total 20 columns):
v1       2310 non-null float64
v2       2310 non-null float64
v3       2310 non-null int64
v4       2310 non-null float64
v5       2310 non-null float64
v6       2310 non-null float64
v7       2310 non-null float64
v8       2310 non-null float64
v9       2310 non-null float64
v10      2310 non-null float64
v11      2310 non-null float64
v12      2310 non-null float64
v13      2310 non-null float64
v14      2310 non-null float64
v15      2310 non-null float64
v16      2310 non-null float64
v17      2310 non-null float64
v18      2310 non-null float64
v19      2310 non-null float64
class    2310 non-null int64
dtypes: float64(18), int64(2)
memory usage: 361.0 KB


Todos las variables de segment son cuantitativas.

In [20]:
#simulando un 5% de celdas missing
import random
seg5=seg.copy()
seg5= seg5.stack().sample(frac=0.95).unstack().reindex(index=seg5.index, columns=seg5.columns)


In [21]:
seg5.head()

,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14,v15,v16,v17,v18,v19,class
0,218.0,178.0,9.0,0.111111,0.0,0.833333,0.547722,NaN,0.544331,59.629630,52.444443,75.222220,51.222220,-21.555555,46.77778,-25.222221,75.222220,0.318996,-2.040554,6.0
1,113.0,130.0,9.0,0.000000,0.0,0.277778,0.250924,0.333333,0.365148,0.888889,0.000000,2.555556,0.111111,-2.666667,5.00000,-2.333333,2.555556,1.000000,-2.123254,3.0
2,202.0,41.0,9.0,0.000000,0.0,0.944448,0.772202,1.111112,1.025597,123.037040,111.888885,139.777790,117.444440,-33.444443,50.22222,-16.777779,139.777790,0.199347,-2.299918,2.0
3,32.0,173.0,9.0,0.000000,0.0,1.722222,1.781593,9.000000,6.749488,43.592594,39.555557,52.888890,38.333336,-12.111111,27.88889,-15.777778,52.888890,0.266914,-1.998858,6.0
4,61.0,197.0,9.0,NaN,NaN,1.444444,1.515353,2.611111,1.925463,49.592594,44.222220,61.555557,43.000000,-16.111110,35.88889,-19.777779,61.555557,NaN,-2.022274,6.0


In [22]:
seg5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2310 entries, 0 to 2309
Data columns (total 20 columns):
v1       2201 non-null float64
v2       2180 non-null float64
v3       2177 non-null float64
v4       2183 non-null float64
v5       2194 non-null float64
v6       2195 non-null float64
v7       2194 non-null float64
v8       2211 non-null float64
v9       2183 non-null float64
v10      2178 non-null float64
v11      2210 non-null float64
v12      2200 non-null float64
v13      2199 non-null float64
v14      2196 non-null float64
v15      2203 non-null float64
v16      2211 non-null float64
v17      2198 non-null float64
v18      2176 non-null float64
v19      2207 non-null float64
class    2194 non-null float64
dtypes: float64(20)
memory usage: 361.0 KB


In [23]:
#imputando por la media
seg5.mean=seg5.apply(lambda x: x.fillna(x.mean()),axis=0)

In [24]:
#Imputando por la mediana
seg5.median=seg5.apply(lambda x: x.fillna(x.median()),axis=0)

In [25]:
#imputando usando knn
from sklearn import datasets
from sklearn import neighbors

class ImputeLearn(object):
    def __init__(self, mat):
        self.mat = np.array(mat)
        self.mat_isnan = np.isnan(self.mat)
        self.w = np.where(np.isnan(self.mat))

    def impute(self, learner):
        ximp = self.mat.copy()
        for i in np.arange(0, len(self.w[0])):
            n = self.w[0][i] # row where the nan is
            p = self.w[1][i] # column where the nan is
            col_isnan = self.mat_isnan[n, :] # empty columns in row n
            train = np.delete(self.mat, n, axis = 0) # remove row n to obtain a training set
            train_nonan = train[~np.apply_along_axis(np.any, 1, np.isnan(train)), :] # remove rows where there is a nan in the training set
            target = train_nonan[:, p] # vector to be predicted
            feature = train_nonan[:, ~col_isnan] # matrix of predictors
            learn = learner.fit(feature, target) # learner
            ximp[n, p] = learn.predict(self.mat[n, ~col_isnan].reshape(1, -1)) # predict and replace
        
        return(ximp)

In [26]:
b1=np.array(seg5)

In [27]:
data_imp = ImputeLearn(b1).impute(learner = neighbors.KNeighborsRegressor(n_neighbors = 5))

In [28]:
completoknn5=pd.DataFrame(data_imp)

In [29]:
completoknn5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2310 entries, 0 to 2309
Data columns (total 20 columns):
0     2310 non-null float64
1     2310 non-null float64
2     2310 non-null float64
3     2310 non-null float64
4     2310 non-null float64
5     2310 non-null float64
6     2310 non-null float64
7     2310 non-null float64
8     2310 non-null float64
9     2310 non-null float64
10    2310 non-null float64
11    2310 non-null float64
12    2310 non-null float64
13    2310 non-null float64
14    2310 non-null float64
15    2310 non-null float64
16    2310 non-null float64
17    2310 non-null float64
18    2310 non-null float64
19    2310 non-null float64
dtypes: float64(20)
memory usage: 361.0 KB


In [30]:
from sklearn.metrics import mean_squared_error
#Calculando el cuadrado medio del error para imputacion por la media
completo5mean=pd.DataFrame(seg5.mean)
mean_squared_error(seg,completo5mean)

57.315818938015127

In [31]:
#Calculando el cuadrado medio del error para imputacion por la mediana
completo5median=pd.DataFrame(seg5.median)
mean_squared_error(seg,completo5median)

61.287280661252964

In [33]:
#Calculando el cuadrado medio del error para imputacion knn
mean_squared_error(seg,completoknn5)

25.325704044158805

Notar que el mejor metodo de imputacion ha sido knn

In [45]:
#simulando un 20% de celdas missing
seg20=seg.copy()
seg20= seg20.stack().sample(frac=0.80).unstack().reindex(index=seg20.index, columns=seg20.columns)
seg20.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2310 entries, 0 to 2309
Data columns (total 20 columns):
v1       1834 non-null float64
v2       1856 non-null float64
v3       1862 non-null float64
v4       1847 non-null float64
v5       1830 non-null float64
v6       1838 non-null float64
v7       1833 non-null float64
v8       1820 non-null float64
v9       1872 non-null float64
v10      1843 non-null float64
v11      1858 non-null float64
v12      1858 non-null float64
v13      1836 non-null float64
v14      1845 non-null float64
v15      1846 non-null float64
v16      1858 non-null float64
v17      1871 non-null float64
v18      1852 non-null float64
v19      1846 non-null float64
class    1855 non-null float64
dtypes: float64(20)
memory usage: 361.0 KB


In [46]:
#hallando el numero de filas que tienen missings
sum(seg20.apply(lambda x: sum(x.isnull().values), axis = 1)>0)

2289

In [47]:
#imputando por la media al 20%
seg20.mean=seg20.apply(lambda x: x.fillna(x.mean()),axis=0)
completo20mean=pd.DataFrame(seg20.mean)
mean_squared_error(seg,completo20mean)

304.12354755148914

In [48]:
#imputando por la mediana al 20%
seg20.median=seg20.apply(lambda x: x.fillna(x.median()),axis=0)
completo20median=pd.DataFrame(seg20.median)
mean_squared_error(seg,completo20median)

318.01878624849377

In [49]:
b2=np.array(seg20)

In [50]:
data_imp = ImputeLearn(b2).impute(learner = neighbors.KNeighborsRegressor(n_neighbors = 5))

In [53]:
completoknn20=pd.DataFrame(data_imp)
mean_squared_error(seg,completoknn20)

241.12667520644851

Nuevamente imputacion por knn resulto ser la mejor